In [1]:
import os
import sys
import h5py
import logging

import torch
import numpy as np
import pandas as pd
import scipy
import scanpy as sc
import warnings
from collections import Counter

from torch_sparse import SparseTensor
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
from rpy2.rinterface import RRuntimeWarning

warnings.filterwarnings("ignore", category=RRuntimeWarning)
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
%load_ext rpy2.ipython

# Camp

In [ ]:
dir_path = './raw_data/Camp'
data_path = os.path.join(dir_path, 'Bladder_rm.batch_dge.txt')
label_path = os.path.join(dir_path, 'MCA_CellAssignments.csv')

In [ ]:
%%R

x1 = read.delim("./raw_data/GSE81252_data.cast.log2.lineage.csv", sep=",", stringsAsFactors=FALSE)
rownames(x1) <- x1[,1]
experiment1 <- x1[,2]
x1 <- x1[,-c(1,2)]
x1 <- t(x1)

x2 = read.delim("./raw_data/GSE81252_data.cast.log2.liverbud.csv", sep=",", stringsAsFactors=FALSE)
rownames(x2) <- x2[,1]
experiment2 <- x2[,2]
assignment2 <- x2[,3]
x2 <- x2[,-c(1,2,3)]
x2 <- t(x2)

### ANNOTATIONS
ann = read.table("/raw_data/GSE81252_Ann.txt", header=T, stringsAsFactors=FALSE)
tmp <- colnames(ann)
tmp <- matrix(unlist(strsplit(tmp, "_")), ncol=2, byrow=T)
fixed_names <- paste(tmp[,2], tmp[,1], sep="_")
colnames(ann) <- fixed_names
ann <- t(ann)
# Check duplicate cells
dups <- which(colnames(x1) %in% colnames(x2))
for (i in colnames(x1)[dups]) {
  if (sum(x2[,colnames(x2) == i] == x1[,colnames(x1) == i]) != 19020) {
    print(i);
  }
} 

x1 <- x1[,-dups]
experiment1 <- experiment1[-dups];
MAT <- cbind(x1,x2);

Stage <- c(experiment1, experiment2)
Type <- c(experiment1, assignment2);
Type[grepl("ih",Type)] <- "immature hepatoblast"
Type[grepl("mh",Type)] <- "mature hepatocyte"
Type[grepl("de",Type)] <- "definitive endoderm"
Type[grepl("EC",Type)] <- "endothelial"
Type[grepl("HE",Type)] <- "hepatic endoderm"
Type[grepl("MC",Type)] <- "mesenchymal stem cell"

Source1 <- rep("iPSC line TkDA3-4", times=length(experiment1))
Source2 <- rep("iPSC line TkDA3-4", times=length(experiment2));
Source2[experiment2 == "huvec"] = "HUVEC"
Source2[grepl("lb", experiment2)] = "liver bud"
Source2[grepl("msc", experiment2)] = "Mesenchymal stem cell"
Source <- c(Source1, Source2)
Age <- Stage
Age[Age == "de"] <- "6 days"
Age[Age == "ipsc"] <- "0 days"
Age[grepl("mh", Age)] <- "21 days"
Age[grepl("he", Age)] <- "8 days"
Age[grepl("lb", Age)] <- "liver bud"
Age[grepl("ih", Age)] <- "14 days"
Age[grepl("msc", Age)] <- "msc"
Batch <- Stage
Batch[grepl("lb1", Batch)] = "3"
Batch[grepl("lb2", Batch)] = "4"
Batch[grepl("lb3", Batch)] = "5"
Batch[grepl("lb4", Batch)] = "6"
Batch[grepl("lb5", Batch)] = "7"
Batch[grepl("lb6", Batch)] = "8"
Batch[grepl("1", Batch)] = "1"
Batch[grepl("2", Batch)] = "2"
Batch[grepl("de", Batch)] = "9"
Batch[grepl("ipsc", Batch)] = "10"
Batch[grepl("huvec", Batch)] = "11"
ANN <- data.frame(Species = rep("Homo sapiens", times=length(Stage)), cell_type1 = Type, Source = Source, age = Age, batch=Batch)
rownames(ANN) <- colnames(MAT)

path <- './raw_data/MAT.csv'
write.csv(MAT, path)

path <- './raw_data/ANN.csv'
write.csv(ANN, path)

In [ ]:
dir_path = './raw_data'
mtx_path = os.path.join(dir_path, 'MAT.csv')
ann_path = os.path.join(dir_path, 'ANN.csv')

In [ ]:
count_data = pd.read_csv(mtx_path, index_col=0)
count_data = count_data.transpose()

In [ ]:
ann_data = pd.read_csv(ann_path, index_col=0)
cell_type = ann_data.loc[:,'cell_type1']

In [ ]:
adata = sc.AnnData(count_data)

adata.obs.index.name = 'cell_id'
adata.var.index.name = 'gene_id'

In [ ]:
label_encoder = LabelEncoder()
cell_type = cell_type.values
cell_type = label_encoder.fit_transform(cell_type)

adata.obs['Group'] = cell_type

In [ ]:
dir_path = './data'
path = os.path.join(dir_path, 'Camp.h5')
adata.write(path)

# Mouse ES Cells

In [4]:
dir_path = './raw_data/Mouse_ES'
d0_path = os.path.join(dir_path, 'GSM1599494_ES_d0_main.csv.bz2')
d2_path = os.path.join(dir_path, 'GSM1599497_ES_d2_LIFminus.csv.bz2')
d4_path = os.path.join(dir_path, 'GSM1599498_ES_d4_LIFminus.csv.bz2')
d7_path = os.path.join(dir_path, 'GSM1599499_ES_d7_LIFminus.csv.bz2')

In [5]:
d0 = pd.read_csv(d0_path, header=None)
d0.rename(columns={0:'gene_id'}, inplace=True)
d0.set_index('gene_id', inplace=True)
num_d0 = d0.shape[1]

d2 = pd.read_csv(d2_path, header=None)
d2.rename(columns={0:'gene_id'}, inplace=True)
d2.set_index('gene_id', inplace=True)
num_d2 = d2.shape[1]

d4 = pd.read_csv(d4_path, header=None)
d4.rename(columns={0:'gene_id'}, inplace=True)
d4.set_index('gene_id', inplace=True)
num_d4 = d4.shape[1]

d7 = pd.read_csv(d7_path, header=None)
d7.rename(columns={0:'gene_id'}, inplace=True)
d7.set_index('gene_id', inplace=True)
num_d7 = d7.shape[1]

df = pd.concat([d0, d2, d4, d7], axis=1)
df = df.transpose()

In [8]:
adata = sc.AnnData(df)

cell_id = [f'Cell_{i+1}' for i in range(len(df))]

d0_label = [1] * num_d0
d2_label = [2] * num_d2
d4_label = [3] * num_d4
d7_label = [4] * num_d7

label = np.concatenate((d0_label, d2_label, d4_label, d7_label))

cell_info = pd.DataFrame({'cell_id' : cell_id, 'Group' : label})
cell_info.set_index('cell_id', inplace=True)

adata.obs = cell_info

sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_counts=1)

/home/junseok/anaconda3/envs/pyg2/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [ ]:
path = './data/Mouse_ES.h5'
adata.write(path)

# Mouse bladder cells

In [2]:
dir_path = './raw_data/MCA/'
data_path = os.path.join(dir_path, 'Bladder_rm.batch_dge.txt')
label_path = os.path.join(dir_path, 'MCA_CellAssignments.csv')

In [3]:
df = pd.read_csv(data_path, sep=' ')
df = df.transpose()
df.index.name = 'cell_id'
df.columns.name = 'gene_id'
adata = sc.AnnData(df)

label_df = pd.read_csv(label_path, index_col=0)
label_df = label_df[:2746]

In [4]:
cell_name = np.array(label_df['Cell.name'])
adata = adata[cell_name,:]
cluster_id = np.array(label_df['ClusterID'])
group = list(map(lambda x : x[8:], cluster_id))
group = np.array(group).astype(int)
adata.obs['Group'] = group

sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_counts=1)

Trying to set attribute `.obs` of view, copying.


In [5]:
path = './data/MCA.h5'
adata.write(path)

# Zeisel

In [6]:
name='Zeisel'
dir_path = './raw_data/Zeisel'
data_path = os.path.join(dir_path, 'GSE60361_C1-3005-Expression.txt.gz')
label_path = os.path.join(dir_path, 'expression_mRNA_17-Aug-2014.txt')

In [7]:
df = pd.read_csv(data_path, sep='\t')
df.rename(columns={'cell_id':'gene_id'}, inplace=True)
df.set_index('gene_id', inplace=True)
df = df.transpose()

adata = sc.AnnData(df)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [8]:
f = open(label_path, 'r')
line = f.readline() # tissue
group = f.readline() # group

group = group[:-1].split(sep='\t')[2:]
group = list(map(int, group))

In [9]:
cell_id = df.index
adata.obs.index.name = 'cell_id'
adata.obs['Group'] = group

adata.var_names_make_unique()
sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_counts=1)

In [10]:
path = './data/Zeisel.h5'
adata.write(path)

# Worm neuron cells

In [ ]:
%%R -o X -o cell_id -o gene_id -o pData

suppressPackageStartupMessages({
  library(monocle)
  library(dplyr)
  library(ggplot2)
})

path = './raw_data/Worm_neuron_cells/Cao_et_al_2017_vignette.RData'

download.file(
    "http://waterston.gs.washington.edu/sci_RNA_seq_gene_count_data/Cao_et_al_2017_vignette.RData",
     destfile=path )

load(path)

expression <- exprs(cds.neurons)
X <- as.matrix(expression)

gene_id <- expression@Dimnames[[1]]
cell_id <- expression@Dimnames[[2]]

pData <- pData(cds.neurons)


In [24]:
cell_id = np.array(cell_id)
gene_id = np.array(gene_id)

label_df = pData[pData.loc[:,'cell.type'] != 'Unclassified neurons']
label_df = label_df[~label_df.loc[:, 'cell.type'].isnull()]

mapping_dict ={'Canal associated neurons' : 1,
        'Cholinergic neurons' : 2,
        'Ciliated sensory neurons' : 3,
        'Dopaminergic neurons' : 4,
        'GABAergic neurons' : 5,
        'Other interneurons' : 6,
        'Oxygen sensory neurons' : 7,
        'Pharyngeal neurons' : 8,
        'Touch receptor neurons' : 9,        
        'flp-1(+) interneurons' : 10,
}

label_df['cell.type'].replace(mapping_dict, inplace=True)

In [25]:
adata = sc.AnnData(X.transpose())

adata.obs.index = cell_id
adata.var.index = gene_id

adata.obs.index.name = 'cell_id'
adata.var.index.name = 'gene_id'

cell = np.array(label_df['cell'])
cell_type = np.array(label_df['cell.type'])

adata = adata[cell]
adata.obs['Group'] = cell_type

sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_counts=1)

Trying to set attribute `.obs` of view, copying.


In [ ]:
path = './data/Worm_neuron_cells'
adata.write(path)

# 10X PBMC

In [27]:
dir_path = './raw_data/10X_PBMC/'
data_path = os.path.join(dir_path, 'filtered_gene_bc_matrices', 'GRCh38')
label_path = os.path.join(dir_path, 'analysis', 'clustering', 'graphclust', 'clusters.csv')

In [28]:
mtx_path = os.path.join(data_path, 'matrix.mtx')
barcode_path = os.path.join(data_path, 'barcodes.tsv')
genes_path = os.path.join(data_path, 'genes.tsv')

In [29]:
adata = sc.read(mtx_path, cache=True)
barcodes = pd.read_csv(barcode_path, header=None, sep='\t')
genes = pd.read_csv(genes_path, header=None, sep='\t')

adata = adata.transpose()
adata.X = adata.X.toarray()

barcodes.rename(columns={0:'barcode'}, inplace=True)
barcodes.set_index('barcode', inplace=True)
adata.obs = barcodes

genes.rename(columns={0:'gene_id', 1:'gene_symbol'}, inplace=True)
genes.set_index('gene_symbol', inplace=True)
adata.var = genes
adata.var_names_make_unique()

label = pd.read_csv(label_path)
label.set_index('Barcode', inplace=True)
group = np.array(label.loc[:,'Cluster'])

adata.obs.loc[label.index, 'Group'] = group

sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_counts=1)

In [30]:
path = './data/10X_PBMC.h5'
adata.write(path)

AnnData object with n_obs × n_vars = 4340 × 19773
    obs: 'Group', 'n_counts'
    var: 'gene_id', 'n_counts'

# Human kidney cells

In [31]:
dir_path = './raw_data/Human_kidney_cells'
data_path = os.path.join(dir_path, 'data.h5')

In [32]:
f = h5py.File(data_path)

expression = f['exprs']
obs = f['obs']
cell_type = np.array(f['obs']['cell_type1'])
obs_names = np.array(f['obs_names'])
var_names = np.array(f['var_names'])

In [33]:
data = np.array(f['exprs']['data'])
indices = np.array(f['exprs']['indices'])
indptr = np.array(f['exprs']['indptr'])
shape = np.array(f['exprs']['shape'])

In [34]:
csr_mat = scipy.sparse.csr_matrix((data, indices, indptr), shape=shape)
X = csr_mat.toarray()

label_encoder = LabelEncoder()
group = label_encoder.fit_transform(cell_type)

In [35]:
adata = sc.AnnData(X)
adata.obs_names = obs_names
adata.var_names = var_names
adata.obs['Group'] = group

sc.pp.filter_genes(adata, min_counts=1)
sc.pp.filter_cells(adata, min_counts=1)

In [38]:
path = './data/Human_kidney_cells.h5'
adata.write(path)

# Baron

In [ ]:
%%R

h1 <- read.csv("./raw_data/GSM2230757_human1_umifm_counts.csv", header = T)
rownames(h1) <- h1[,1]
labels_h1 <- as.character(h1$assigned_cluster)
h1 <- h1[,4:ncol(h1)]
h1 <- t(h1)
# human2
h2 <- read.csv("./raw_data/GSM2230758_human2_umifm_counts.csv", header = T)
rownames(h2) <- h2[,1]
labels_h2 <- as.character(h2$assigned_cluster)
h2 <- h2[,4:ncol(h2)]
h2 <- t(h2)
# human3
h3 <- read.csv("./raw_data/GSM2230759_human3_umifm_counts.csv", header = T)
rownames(h3) <- h3[,1]
labels_h3 <- as.character(h3$assigned_cluster)
h3 <- h3[,4:ncol(h3)]
h3 <- t(h3)
# human4
h4 <- read.csv("./raw_data/GSM2230760_human4_umifm_counts.csv", header = T)
rownames(h4) <- h4[,1]
labels_h4 <- as.character(h4$assigned_cluster)
h4 <- h4[,4:ncol(h4)]
h4 <- t(h4)

### ANNOTATIONS
# human
h_ann <- data.frame(
  human = c(
    rep(1, length(labels_h1)), 
    rep(2, length(labels_h2)), 
    rep(3, length(labels_h3)), 
    rep(4, length(labels_h4))
  ),
  cell_type1 = c(labels_h1, labels_h2, labels_h3, labels_h4))
rownames(h_ann) <- colnames(h)


path <- './raw_data/h.csv'
write.csv(h, path)

path <- './raw_data/h_ann.csv'
write.csv(h_ann, path)

In [ ]:
name='Baron'
dir_path = '../data/Baron'
mtx_path = os.path.join(dir_path, 'h.csv')
ann_path = os.path.join(dir_path, 'h_ann.csv')

In [ ]:
count_data = pd.read_csv(mtx_path, index_col=0)
count_data = count_data.transpose()

ann_data = pd.read_csv(ann_path, index_col=0)
cell_type = ann_data.loc[:,'cell_type1']

In [ ]:
adata = sc.AnnData(count_data)

adata.obs.index.name = 'cell_id'
adata.var.index.name = 'gene_id'

In [ ]:
label_encoder = LabelEncoder()
cell_type = cell_type.values
cell_type = label_encoder.fit_transform(cell_type)

adata.obs['Group'] = cell_type

In [ ]:
dir_path = './data/Baron_human'
path = os.path.join(dir_path, 'Baron_human.h5')
adata.write(path)

# Shekhar mouse retina cells

In [ ]:
%%R -o X -o cell_id -o gene_id -o pData

### DATA
path <- './raw_data/bipolar_data_Cell2016.Rdata'
load(path)
# Remove libraries that contain more than 10% mitochondrially derived transcripts
mt.genes = grep("mt-", rownames(bipolar_dge), value = TRUE)
cells.use = colnames(bipolar_dge)[colSums(bipolar_dge[mt.genes, ])/colSums(bipolar_dge) < 0.1]
bipolar_dge = bipolar_dge[, cells.use]

# Initialize single cell data as an S4 class object. Only cells where > 500 
# genes are detected are considered.
# Among the selected cells, only genes that are present in > 30 cells and 
# those having > 60 transcripts summed
# across all the selected cells are considered.
bipolar_dge <- bipolar_dge[ , colSums(bipolar_dge > 0) > 500]
bipolar_dge <- bipolar_dge[rowSums(bipolar_dge > 0) > 30 & rowSums(bipolar_dge) > 60, ]


### ANNOTATIONS
# use cluster file from https://portals.broadinstitute.org/single_cell/study/retinal-bipolar-neuron-drop-seq
path <- './raw_data/clust_retinal_bipolar.txt' 
d <- read.table(path, sep = "\t", header = T)
cell_ids <- d[,1]
d <- data.frame(cell_type2 = d[,2])
rownames(d) <- cell_ids
# annotation louvain clusters (using Fig.1,F from the paper)
d$clust_id <- NA
d$clust_id[d$cell_type2 == "BC1A"] <- 7
d$clust_id[d$cell_type2 == "BC1B"] <- 9
d$clust_id[d$cell_type2 == "BC2"] <- 10
d$clust_id[d$cell_type2 == "BC3A"] <- 12
d$clust_id[d$cell_type2 == "BC3B"] <- 8
d$clust_id[d$cell_type2 == "BC4"] <- 14
d$clust_id[d$cell_type2 == "BC5A (Cone Bipolar cell 5A)"] <- 3
d$clust_id[d$cell_type2 == "BC5B"] <- 13
d$clust_id[d$cell_type2 == "BC5C"] <- 6
d$clust_id[d$cell_type2 == "BC5D"] <- 11
d$clust_id[d$cell_type2 == "BC6"] <- 5
d$clust_id[d$cell_type2 == "BC7 (Cone Bipolar cell 7)"] <- 4
d$clust_id[d$cell_type2 == "BC8/9 (mixture of BC8 and BC9)"] <- 15
d$clust_id[d$cell_type2 == "RBC (Rod Bipolar cell)"] <- 1
d$clust_id[d$cell_type2 == "MG (Mueller Glia)"] <- 2
d$clust_id[d$cell_type2 == "AC (Amacrine cell)"] <- 16
d$clust_id[d$cell_type2 == "Rod Photoreceptors"] <- 20
d$clust_id[d$cell_type2 == "Cone Photoreceptors"] <- 22
# our manual annotation
d$cell_type1 <- "unknown"
d$cell_type1[grepl("BC", d$cell_type2)] <- "bipolar"
d$cell_type1[grepl("MG", d$cell_type2)] <- "muller"
d$cell_type1[grepl("AC", d$cell_type2)] <- "amacrine"
d$cell_type1[grepl("Rod Photoreceptors", d$cell_type2)] <- "rods"
d$cell_type1[grepl("Cone Photoreceptors", d$cell_type2)] <- "cones"

path <- './raw_data/Shekhar_counts.csv'
write.csv(bipolar_dge, path)

path <- './raw_data/colData.csv'
write.csv(d, path)


In [39]:
dir_path = './raw_data'
mtx_path = os.path.join(dir_path, 'Shekhar_counts.csv')
col_path = os.path.join(dir_path, 'colData.csv')

In [40]:
count_data = pd.read_csv(mtx_path, index_col=0)
count_data = count_data.transpose()

col_data = pd.read_csv(col_path, index_col=0)[1:]
cell_type = col_data.iloc[:,0]

In [ ]:
adata = sc.AnnData(count_data)

adata.obs.index.name = 'cell_id'
adata.var.index.name = 'gene_id'

label_encoder = LabelEncoder()
cell_type = cell_type.values
cell_type = label_encoder.fit_transform(cell_type)

adata.obs['Group'] = cell_type


In [ ]:
dir_path = '../data/Shekhar'
path = os.path.join(dir_path, 'Shekhar.h5')
adata.write(path)

In [ ]:
dir_path = './raw_data'
mtx_path = os.path.join(dir_path, 'Shekhar_counts.csv')
col_path = os.path.join(dir_path, 'colData.csv')

count_data = pd.read_csv(mtx_path, index_col=0)
count_data = count_data.transpose()

col_data = pd.read_csv(col_path, index_col=0)[1:]
cell_type = col_data.iloc[:,0]

adata = sc.AnnData(count_data)

adata.obs.index.name = 'cell_id'
adata.var.index.name = 'gene_id'

label_encoder = LabelEncoder()
cell_type = cell_type.values
cell_type = label_encoder.fit_transform(cell_type)

adata.obs['Group'] = cell_type

dir_path = './data/Shekhar'
path = os.path.join(dir_path, 'Shekhar.h5')
adata.write(path)